In [1]:
import pandas as pd
import os
import wrangle
import matplotlib as plt
import seaborn as sns
import numpy as np
from env import host, user, password
from wrangle import wrangle_zillow
import pandas as pd
import env
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_connection(db, username=user, host=host, password=password):
    return f'mysql+pymysql://{username}:{password}@{host}/{db}'

## Aquiring the data

In [3]:
sql = """
        SELECT *
        FROM properties_2017
        LEFT JOIN predictions_2017 AS pe USING (parcelid)
        LEFT JOIN architecturalstyletype AS arch USING (architecturalstyletypeid)
        LEFT JOIN propertylandusetype USING (propertylandusetypeid)
        LEFT JOIN airconditioningtype USING (airconditioningtypeid)
        LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
        LEFT JOIN storytype USING (storytypeid)
        LEFT JOIN unique_properties USING (parcelid)
        LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
        WHERE propertylandusetype.propertylandusedesc = 'Single Family Residential'
        AND pe.transactiondate LIKE '2017%%';
    """

df = pd.read_sql(sql, get_connection("zillow"));

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   heatingorsystemtypeid         33935 non-null  float64
 1   parcelid                      52441 non-null  int64  
 2   storytypeid                   47 non-null     float64
 3   typeconstructiontypeid        76 non-null     float64
 4   airconditioningtypeid         13638 non-null  float64
 5   propertylandusetypeid         52441 non-null  float64
 6   architecturalstyletypeid      70 non-null     float64
 7   id                            52441 non-null  int64  
 8   basementsqft                  47 non-null     float64
 9   bathroomcnt                   52441 non-null  float64
 10  bedroomcnt                    52441 non-null  float64
 11  buildingclasstypeid           0 non-null      object 
 12  buildingqualitytypeid         33740 non-null  float64
 13  c

In [5]:
df.head()

,heatingorsystemtypeid,parcelid,storytypeid,typeconstructiontypeid,airconditioningtypeid,propertylandusetypeid,architecturalstyletypeid,id,basementsqft,bathroomcnt,...,censustractandblock,id,logerror,transactiondate,architecturalstyledesc,propertylandusedesc,airconditioningdesc,typeconstructiondesc,storydesc,heatingorsystemdesc
0,NaN,14297519,NaN,NaN,NaN,261.0,NaN,1727539,NaN,3.5,...,6.059063e+13,0,0.025595,2017-01-01,None,Single Family Residential,None,None,None,None
1,NaN,17052889,NaN,NaN,NaN,261.0,NaN,1387261,NaN,1.0,...,6.111001e+13,1,0.055619,2017-01-01,None,Single Family Residential,None,None,None,None
2,NaN,14186244,NaN,NaN,NaN,261.0,NaN,11677,NaN,2.0,...,6.059022e+13,2,0.005383,2017-01-01,None,Single Family Residential,None,None,None,None
3,2.0,12177905,NaN,NaN,NaN,261.0,NaN,2288172,NaN,3.0,...,6.037300e+13,3,-0.103410,2017-01-01,None,Single Family Residential,None,None,None,Central
4,2.0,12095076,NaN,NaN,1.0,261.0,NaN,781532,NaN,3.0,...,6.037461e+13,6,-0.001011,2017-01-01,None,Single Family Residential,Central,None,None,Central


In [6]:
pd.set_option('display.max_columns', None)

df.head()

,heatingorsystemtypeid,parcelid,storytypeid,typeconstructiontypeid,airconditioningtypeid,propertylandusetypeid,architecturalstyletypeid,id,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate,architecturalstyledesc,propertylandusedesc,airconditioningdesc,typeconstructiondesc,storydesc,heatingorsystemdesc
0,NaN,14297519,NaN,NaN,NaN,261.0,NaN,1727539,NaN,3.5,4.0,None,NaN,3.5,NaN,NaN,3100.0,3100.0,None,None,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,NaN,122,None,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,1.0,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,None,NaN,6.059063e+13,0,0.025595,2017-01-01,None,Single Family Residential,None,None,None,None
1,NaN,17052889,NaN,NaN,NaN,261.0,NaN,1387261,NaN,1.0,2.0,None,NaN,1.0,NaN,1465.0,1465.0,1465.0,None,None,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,NaN,1110,None,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,None,NaN,6.111001e+13,1,0.055619,2017-01-01,None,Single Family Residential,None,None,None,None
2,NaN,14186244,NaN,NaN,NaN,261.0,NaN,11677,NaN,2.0,3.0,None,NaN,2.0,NaN,NaN,1243.0,1243.0,None,None,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,1.0,122,None,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,None,NaN,6.059022e+13,2,0.005383,2017-01-01,None,Single Family Residential,None,None,None,None
3,2.0,12177905,NaN,NaN,NaN,261.0,NaN,2288172,NaN,3.0,4.0,None,8.0,3.0,NaN,NaN,2376.0,2376.0,None,None,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,1.0,0101,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,None,NaN,6.037300e+13,3,-0.103410,2017-01-01,None,Single Family Residential,None,None,None,Central
4,2.0,12095076,NaN,NaN,1.0,261.0,NaN,781532,NaN,3.0,4.0,None,9.0,3.0,NaN,NaN,2962.0,2962.0,None,None,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34145202.0,-118179824.0,63000.0,1.0,NaN,NaN,NaN,1.0,0101,PSR2,6.037461e+07,47019.0,3101.0,274684.0,96293.0,0.0,NaN,1.0,NaN,NaN,1950.0,NaN,NaN,276684.0,773303.0,2016.0,496619.0,9516.26,None,NaN,6.037461e+13,6,-0.001011,2017-01-01,None,Single Family Residential,Central,None,None,Central


In [ ]:
df.shape

## Nulls of rows

In [ ]:

nulls_col = pd.DataFrame({'num_rows_missing': df.isnull().sum(), 
              'percent_rows_missing': (df.isnull().sum() / df.shape[0] * 100)})
pd.set_option('display.max_columns', None)
nulls_col.sort_values(by='num_rows_missing', ascending=False).head(10)

In [ ]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing.sort_values(by='num_rows_missing', ascending=True)


In [ ]:
Number of 

In [ ]:
nulls_by_col(df)

## Nulls of collums 

In [ ]:
nulls_row = pd.DataFrame({'num_cols_missing': df.isnull().sum(axis=1),
              'percent_cols_missing': df.isnull().sum(axis=1)/df.shape[1]*100})

nulls_row

- honestly there is so much missing in my cluster excercizez imma keep it simple and just do what i did and keep 60% of the data 

In [ ]:
def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prcnt_miss})
    rows_missing = df.merge(rows_missing,
                        left_index=True,
                        right_index=True)[['parcelid', 'num_cols_missing', 'percent_cols_missing']]
    return rows_missing.sort_values(by='num_cols_missing', ascending=False)

In [ ]:
def summarize(df):
    '''
    summarize will take in a single argument (a pandas dataframe) 
    and output to console various statistics on said dataframe, including:
    # .head()
    # .info()
    # .describe()
    # .value_counts()
    # observation of nulls in the dataframe
    '''
    print('SUMMARY REPORT')
    print('=====================================================\n\n')
    print('Dataframe head: ')
    print(df.head(3))
    print('=====================================================\n\n')
    print('Dataframe info: ')
    print(df.info())
    print('=====================================================\n\n')
    print('Dataframe Description: ')
    print(df.describe())
    num_cols = [col for col in df.columns if df[col].dtype != 'O']
    cat_cols = [col for col in df.columns if col not in num_cols]
    print('=====================================================')
    print('DataFrame value counts: ')
    for col in df.columns:
        if col in cat_cols:
            print(df[col].value_counts(), '\n')
        else:
            print(df[col].value_counts(bins=10, sort=False), '\n')
    print('=====================================================')
    print('nulls in dataframe by column: ')
    print(nulls_by_col(df))
    print('=====================================================')
    print('nulls in dataframe by row: ')
    print(nulls_by_row(df))
    print('=====================================================')


In [ ]:
summarize(df)


## now im going to drop collumns and rows

In [7]:
def remove_columns(df, cols_to_remove):
    df = df.drop(columns=cols_to_remove)
    return df

In [8]:
def handle_missing_values(df, prop_required_columns=0.75, prop_required_row=0.75):
    threshold = int(round(prop_required_columns * len(df.index), 0))
    df = df.dropna(axis=1, thresh=threshold) #1, or ‘columns’ : Drop columns which contain missing value
    threshold = int(round(prop_required_row * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=threshold) #0, or ‘index’ : Drop rows which contain missing values.
    return df

In [9]:
# def drop_r_nulls(df):
#     df = df.dropna()
#     return df

In [10]:
def data_prep(df, cols_to_remove=[], prop_required_column=0.6, prop_required_row=0.75):
    df = remove_columns(df, cols_to_remove)
    df = handle_missing_values(df, prop_required_column, prop_required_row)
    #df = drop_r_nulls(df)
    return df

In [11]:
df = data_prep(df)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52376 entries, 0 to 52440
Data columns (total 35 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   heatingorsystemtypeid         33935 non-null  float64
 1   parcelid                      52376 non-null  int64  
 2   propertylandusetypeid         52376 non-null  float64
 3   id                            52376 non-null  int64  
 4   bathroomcnt                   52376 non-null  float64
 5   bedroomcnt                    52376 non-null  float64
 6   buildingqualitytypeid         33740 non-null  float64
 7   calculatedbathnbr             52304 non-null  float64
 8   calculatedfinishedsquarefeet  52359 non-null  float64
 9   finishedsquarefeet12          52194 non-null  float64
 10  fips                          52376 non-null  float64
 11  fullbathcnt                   52304 non-null  float64
 12  latitude                      52376 non-null  float64
 13  l

In [ ]:
#collumns  dont need = buildingqualitytypeid,finishedsquarefeet12, fullbathcnt, heatingorsystemtypeid, propertyzoningdesc, heatingorsystemdesc,   

In [13]:
df.fips.value_counts()

6037.0    33899
6059.0    14094
6111.0     4383
Name: fips, dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.buildingqualitytypeid.value_counts()

In [ ]:
data_prep(df).to_csv('zillow.csv')

In [ ]:
df2 = pd.read_csv('zillow.csv')
df2.head()

In [ ]:
df2.shape

In [ ]:
df.info()

In [ ]:
get_zillow_data(df)

In [ ]:
df = wrangle_zillow()

In [ ]:
df.head()

In [ ]:
def drop_one(df):
    df = df.drop(columns=['Unnamed: 0'])
    return df
    

In [ ]:
drop_one(df)


In [ ]:
df = df.drop(columns=['Unnamed: 0'])



 -----------------------

In [ ]:
#aquiring the data 

In [ ]:
df = wrangle_zillow()

In [ ]:
df.head()

In [ ]:
df.fips.value_counts()

In [ ]:
def fix_fips(df):
    df['County'] = df.fips.map({6037:'Los Angeles County', 6059:'Orange County', 6111:'Ventura County'})
    return df

In [ ]:
 df = fix_fips(df)

In [ ]:
df.head()

In [ ]:
 df['age'] = 2017 - df.yearbuilt

In [ ]:
df.head()

In [ ]:
df['age_bin'] = pd.cut(df.age, 
                           bins = [0, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140],
                           labels = [0, .066, .133, .20, .266, .333, .40, .466, .533, 
                                     .60, .666, .733, .8, .866, .933])


In [ ]:
df.head()

In [ ]:
df['acres'] = df.lotsizesquarefeet/43560

In [ ]:
 df['acres_bin'] = pd.cut(df.acres, bins = [0, .10, .15, .25, .5, 1, 5, 10, 20, 50, 200], 
                       labels = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9])

In [ ]:
df.head()

In [ ]:
df.County.value_counts()

In [ ]:
def get_counties():
    '''
    This function will create dummy variables out of the original fips column. 
    And return a dataframe with all of the original columns except regionidcounty.
    We will keep fips column for data validation after making changes. 
    New columns added will be 'LA', 'Orange', and 'Ventura' which are boolean 
    The fips ids are renamed to be the name of the county each represents. 
    '''
    # create dummy vars of fips id
    county_df = pd.get_dummies(df.fips)
    # rename columns by actual county name
    county_df.columns = ['LA', 'Orange', 'Ventura']
    # concatenate the dataframe with the 3 county columns to the original dataframe
    df_dummies = pd.concat([df, county_df], axis = 1)
    # drop regionidcounty and fips columns
    df_dummies = df_dummies.drop(columns = ['regionidcounty'])
    return df_dummies


In [ ]:
df = get_counties()
df.head()


In [ ]:
for col in df.columns:
    sns.boxplot(df[col])
    plt.title(col)
    plt.show()
